# Advanced Stock Price Forecasting with LSTM

This notebook provides a comprehensive approach to stock price forecasting using Long Short-Term Memory (LSTM) neural networks. Unlike simple demonstrations, this implementation includes:

- **Parameterized inputs** for any stock ticker and date range
- **Feature engineering** with technical indicators (SMA, RSI)
- **Baseline comparison** to evaluate model effectiveness
- **Critical performance analysis** including directional accuracy
- **Practical insights** and limitations discussion

## Parameters
Modify these parameters to analyze different stocks and time periods:


In [1]:
# Stock Analysis Parameters
STOCK_TICKER = 'MRK'  # Change this to any stock ticker (e.g., 'MSFT', 'GOOGL', 'TSLA')
START_DATE = '2009-01-01'  # Start date for historical data
END_DATE = '2020-12-31'    # End date for historical data

# Model Parameters
SEQUENCE_LENGTH = 30  # Number of days to look back for prediction
TEST_SIZE = 0.2      # Proportion of data for testing
EPOCHS = 50          # Number of training epochs
BATCH_SIZE = 32      # Batch size for training


## 1. Import Required Libraries


In [2]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install yfinance
# !pip install scikit-learn
# !pip install tensorflow
# !pip install pandas_ta
# !pip install scipy-stubs
# !pip install pandas-stubs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
import requests
from datetime import datetime, timedelta

# Machine Learning Libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Set style and suppress warnings
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Analysis target: {STOCK_TICKER} from {START_DATE} to {END_DATE}")


2025-07-22 10:49:36.869633: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 10:49:36.881949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753181376.896439   27390 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753181376.900772   27390 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753181376.911519   27390 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow version: 2.19.0
Analysis target: MRK from 2009-01-01 to 2020-12-31


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [7]:
def save_news_dataframe(df: pd.DataFrame, file_path: str = "stock_data.csv") -> bool:
    """
    Saves the news DataFrame to a CSV file.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        file_path (str): The path to the CSV file.

    Returns:
        bool: True if the save was successful, False otherwise.
    """
    try:
        df.to_csv(file_path, index=False)
        print(f"News DataFrame saved to {file_path}")
        return True
    except Exception as e:
        print(f"Error: Failed to save news DataFrame: {e}")
        return False


## 2. Data Retrieval & Preprocessing

We use yfinance to automatically handle historical price adjustments for stock splits and dividends, ensuring our data reflects true historical performance.


In [8]:
file_name = "MRK_full_sorted_data.csv"

def fetch_static_stock_data():
    """
    Fetch stock data using yfinance with automatic adjustment for splits and dividends
    """
    try:
        # Load the data from the CSV file
        # index_col=0: Sets the first column (our timestamps) as the DataFrame index.
        # parse_dates=True: Converts the index column into datetime objects for time series analysis.
        df = pd.read_csv(file_name, index_col=0, parse_dates=True)

        print(f"✅ Data loaded successfully from '{file_name}'")

        # Display the first few rows to confirm it's loaded correctly
        print("\n--- First 5 Rows ---")
        print(df.head())

        # You can also check the data types to ensure the index is a datetime
        print("\n--- Data Info ---")
        df.info()

        return df

    except FileNotFoundError:
        print(f"❌ Error: The file '{file_name}' was not found.")
        print("Please make sure the CSV file is in the same folder as your script.")

def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch stock data using yfinance with automatic adjustment for splits and dividends
    """
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval='90m')
        # data = stock.h
        # istory(period='max', interval='1d')

        if data.empty:
            raise ValueError(f"No data found for ticker {ticker}")

        print(f"Successfully fetched {len(data)} days of data for {ticker}")
        print(f"Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")

        return data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None


# Fetch the stock data
# stock_data = fetch_stock_data(STOCK_TICKER, START_DATE, END_DATE)
stock_data = fetch_static_stock_data()
print(type(stock_data))

# save_news_dataframe(df=stock_data, file_path="stock_data_nvda.csv")

if stock_data is not None:
    print("\nStock data overview:")
    print(stock_data.head())
    print(f"\nData shape: {stock_data.shape}")
    print(f"Missing values: {stock_data.isnull().sum().sum()}")


✅ Data loaded successfully from 'MRK_full_sorted_data.csv'

--- First 5 Rows ---
                        High      Low    Close  Volume
Date                                                  
2009-01-02 08:21:00  16.2320  16.2320  16.2320     104
2009-01-02 08:36:00  16.2374  16.2320  16.2320     209
2009-01-02 08:38:00  16.2374  16.2320  16.2320     209
2009-01-02 08:39:00  16.2320  16.2320  16.2320     104
2009-01-02 08:46:00  16.3721  16.3721  16.3721    1347

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1238712 entries, 2009-01-02 08:21:00 to 2020-12-31 19:17:00
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   High    1238712 non-null  float64
 1   Low     1238712 non-null  float64
 2   Close   1238712 non-null  float64
 3   Volume  1238712 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 47.3 MB
<class 'pandas.core.frame.DataFrame'>

Stock data overview:
                        H

## 3. Feature Engineering with Technical Indicators

Instead of using only closing prices, we'll engineer additional features that provide more context to our model:

- **7-day Simple Moving Average (SMA)**: Smooths out price fluctuations
- **Relative Strength Index (RSI)**: Measures momentum and identifies overbought/oversold conditions


In [9]:
def calculate_technical_indicators(data):
    """
    Calculate technical indicators for enhanced feature set
    """
    df = data.copy()

    # 7-day Simple Moving Average
    df['SMA_7'] = df['Close'].rolling(window=7).mean()

    # Relative Strength Index (RSI)
    def calculate_rsi(prices, window=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    df['RSI'] = calculate_rsi(df['Close'])

    # Price change percentage
    df['Price_Change_Pct'] = df['Close'].pct_change()

    # Volume moving average
    df['Volume_MA'] = df['Volume'].rolling(window=7).mean()

    # High-Low spread
    df['HL_Spread'] = (df['High'] - df['Low']) / df['Close']

    # # Bollinger Bands
    # df.ta.bbands(close='Close', length=20, append=True)
    #
    # # Moving Average Convergence Divergence (MACD)
    # df.ta.macd(close='Close', append=True)

    return df

# Calculate technical indicators
enhanced_data = calculate_technical_indicators(stock_data)

# Remove rows with NaN values (due to rolling calculations)
enhanced_data = enhanced_data.dropna()

print(f"Enhanced dataset shape: {enhanced_data.shape}")
print("\nNew features added:")
print(enhanced_data[['Close', 'SMA_7', 'RSI', 'Price_Change_Pct', 'HL_Spread']].head(10))


Enhanced dataset shape: (1238686, 9)

New features added:
                       Close      SMA_7        RSI  Price_Change_Pct  \
Date                                                                   
2009-01-02 09:24:00  16.3775  16.301300  72.884555          0.005624   
2009-01-02 09:26:00  16.3775  16.313614  72.884555          0.000000   
2009-01-02 09:30:00  16.4927  16.342386  80.096975          0.007034   
2009-01-02 09:31:00  16.4959  16.371614  80.242952          0.000194   
2009-01-02 09:32:00  16.5336  16.407000  81.814346          0.002285   
2009-01-02 09:33:00  16.5606  16.446243  76.115267          0.001633   
2009-01-02 09:34:00  16.5821  16.488557  85.929794          0.001298   
2009-01-02 09:35:00  16.5888  16.518743  98.248459          0.000404   
2009-01-02 09:36:00  16.5363  16.541429  83.952328         -0.003165   
2009-01-02 09:37:00  16.5444  16.548814  84.304690          0.000490   

                     HL_Spread  
Date                            
2009-01-02 

## 4. Data Visualization


In [10]:
# Visualization# Create comprehensive visualization
# fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# fig.suptitle(f'{STOCK_TICKER} Stock Analysis with Technical Indicators', fontsize=16, fontweight='bold')
#
# # Price and SMA
# axes[0, 0].plot(enhanced_data.index, enhanced_data['Close'], label='Close Price', alpha=0.7)
# axes[0, 0].plot(enhanced_data.index, enhanced_data['SMA_7'], label='7-day SMA', color='red')
# axes[0, 0].set_title('Stock Price with 7-day SMA')
# axes[0, 0].set_ylabel('Price ($)')
# axes[0, 0].legend()
# axes[0, 0].grid(True, alpha=0.3)
#
# # RSI
# axes[0, 1].plot(enhanced_data.index, enhanced_data['RSI'], color='purple')
# axes[0, 1].axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
# axes[0, 1].axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
# axes[0, 1].set_title('Relative Strength Index (RSI)')
# axes[0, 1].set_ylabel('RSI')
# axes[0, 1].legend()
# axes[0, 1].grid(True, alpha=0.3)
#
# # Volume
# axes[1, 0].bar(enhanced_data.index, enhanced_data['Volume'], alpha=0.6, color='orange')
# axes[1, 0].plot(enhanced_data.index, enhanced_data['Volume_MA'], color='red', label='Volume MA')
# axes[1, 0].set_title('Trading Volume')
# axes[1, 0].set_ylabel('Volume')
# axes[1, 0].legend()
# axes[1, 0].grid(True, alpha=0.3)
#
# # Price change distribution
# axes[1, 1].hist(enhanced_data['Price_Change_Pct'].dropna(), bins=50, alpha=0.7, color='green')
# axes[1, 1].set_title('Daily Price Change Distribution')
# axes[1, 1].set_xlabel('Price Change (%)')
# axes[1, 1].set_ylabel('Frequency')
# axes[1, 1].grid(True, alpha=0.3)
#
# plt.tight_layout()
# plt.show()
#
# # Display correlation matrix
# features_for_correlation = ['Close', 'SMA_7', 'RSI', 'Volume', 'HL_Spread']
# # features_for_correlation = ['Close', 'SMA_7', 'RSI', 'Volume', 'HL_Spread', 'BBL_20_2.0','MACD_12_26_9']
# correlation_matrix = enhanced_data[features_for_correlation].corr()
#
# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
#             square=True, fmt='.2f')
# plt.title('Feature Correlation Matrix')
# plt.tight_layout()
# plt.show()


## 5. Data Scaling and Preparation

**Why MinMaxScaler is crucial for LSTMs:**

LSTM networks use activation functions like tanh and sigmoid that work optimally with inputs in specific ranges (typically 0-1 or -1 to 1). Without proper scaling:
- **Gradient problems**: Large input values can cause vanishing or exploding gradients
- **Slow convergence**: The network takes longer to learn patterns
- **Poor performance**: Features with larger scales dominate the learning process

MinMaxScaler transforms all features to the same scale (0-1), ensuring equal importance during training.


In [11]:
def prepare_lstm_data(data, sequence_length, test_size):
    """
    Prepare data for LSTM training with multiple features
    """
    # Select features for the model
    feature_columns = ['Close', 'SMA_7', 'RSI', 'Volume', 'HL_Spread']
    # feature_columns = ['Close', 'SMA_7', 'RSI', 'Volume', 'HL_Spread', 'BBL_20_2.0','MACD_12_26_9']
    features = data[feature_columns].values

    # Scale the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler.fit_transform(features)

    # Create sequences
    X, y = [], []
    for i in range(sequence_length, len(scaled_features)):
        X.append(scaled_features[i-sequence_length:i])  # All features for sequence
        y.append(scaled_features[i, 0])  # Only Close price as target

    X, y = np.array(X), np.array(y)

    # Split into train and test sets
    split_index = int(len(X) * (1 - test_size))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    return X_train, X_test, y_train, y_test, scaler, feature_columns

# Prepare the data
X_train, X_test, y_train, y_test, scaler, feature_names = prepare_lstm_data(
    enhanced_data, SEQUENCE_LENGTH, TEST_SIZE
)

print(f"Training data shape: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Testing data shape: X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"Features used: {feature_names}")
print(f"Sequence length: {SEQUENCE_LENGTH} days")


Training data shape: X_train: (990924, 30, 5), y_train: (990924,)
Testing data shape: X_test: (247732, 30, 5), y_test: (247732,)
Features used: ['Close', 'SMA_7', 'RSI', 'Volume', 'HL_Spread']
Sequence length: 30 days


## 6. Train-Test Split Discussion

**Current Approach: 80-20 Split**
We're using a standard 80-20 split where the most recent 20% of data serves as our test set.

**Why Walk-Forward Validation is Often Better for Time Series:**

1. **Temporal Integrity**: Walk-forward validation respects the time-ordered nature of financial data
2. **Realistic Testing**: It simulates real-world trading where you only have past data to predict future prices
3. **Robust Evaluation**: Multiple test periods provide better performance estimates
4. **Overfitting Detection**: Helps identify if the model works consistently across different market conditions

**Implementation Note**: For production systems, consider implementing walk-forward validation with multiple train-test cycles, retraining the model periodically as new data becomes available.


## 7. LSTM Model Architecture


In [12]:
def build_lstm_model(input_shape):
    """
    Build a stacked LSTM model with dropout layers for regularization
    """
    model = Sequential([
        # First LSTM layer with return_sequences=True to stack layers
        LSTM(units=100, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(units=100, return_sequences=True),
        Dropout(0.2),

        # Third LSTM layer (final layer doesn't return sequences)
        LSTM(units=50),
        Dropout(0.2),

        # Dense output layer
        Dense(units=1, activation='linear')
    ])

    # Compile with Adam optimizer and MSE loss
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae']
    )

    return model


# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)

# Display model architecture
print("LSTM Model Architecture:")
model.summary()

# Calculate total parameters
total_params = model.count_params()
print(f"\nTotal trainable parameters: {total_params:,}")


I0000 00:00:1753181382.292473   27390 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 77064 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:00:09.0, compute capability: 9.0


LSTM Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 100)        │        42,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 153,051 (597.86 KB)

 Trainable params: 153,051 (597.86 KB)

 Non-trainable params: 0 (0.00 B)


Total trainable parameters: 153,051


## 8. Model Training with Callbacks


In [ ]:
# Define callbacks for better training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

print(f"Starting training for {EPOCHS} epochs...")
print(f"Training samples: {len(X_train)}, Validation samples: {int(len(X_train) * 0.2)}")

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nTraining completed!")


Starting training for 50 epochs...
Training samples: 990924, Validation samples: 198184
Epoch 1/50


I0000 00:00:1753181385.505939   27529 cuda_dnn.cc:529] Loaded cuDNN version 90501


 5368/24774 ━━━━━━━━━━━━━━━━━━━━ 2:47 9ms/step - loss: 8.3997e-04 - mae: 0.0172

## 9. Training Performance Analysis


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(history.history['loss'], label='Training Loss', color='blue')
ax1.plot(history.history['val_loss'], label='Validation Loss', color='red')
ax1.set_title('Model Loss During Training')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (MSE)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MAE plot
ax2.plot(history.history['mae'], label='Training MAE', color='blue')
ax2.plot(history.history['val_mae'], label='Validation MAE', color='red')
ax2.set_title('Model MAE During Training')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Mean Absolute Error')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check for overfitting
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
overfitting_ratio = final_val_loss / final_train_loss

print(f"Final Training Loss: {final_train_loss:.6f}")
print(f"Final Validation Loss: {final_val_loss:.6f}")
print(f"Overfitting Ratio (Val/Train): {overfitting_ratio:.2f}")

if overfitting_ratio > 1.2:
    print("⚠️  Warning: Model may be overfitting (validation loss > 1.2x training loss)")
elif overfitting_ratio < 1.1:
    print("✅ Good: Model shows minimal overfitting")
else:
    print("✅ Acceptable: Model shows reasonable generalization")


## 10. Model Predictions and Baseline Comparison


In [ ]:
# Make predictions
train_predictions = model.predict(X_train, verbose=0)
test_predictions = model.predict(X_test, verbose=0)

# Create a scaler for inverse transformation (only for Close price)
close_scaler = MinMaxScaler(feature_range=(0, 1))
close_scaler.fit(enhanced_data[['Close']].values)

# Inverse transform predictions and actual values
train_predictions_scaled = close_scaler.inverse_transform(train_predictions)
test_predictions_scaled = close_scaler.inverse_transform(test_predictions.reshape(-1, 1))
y_train_scaled = close_scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_scaled = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Create baseline predictions (naive forecast: tomorrow = today)
# For test set, baseline is the previous day's actual price
baseline_predictions = np.roll(y_test_scaled, 1)
baseline_predictions[0] = y_test_scaled[0]  # Handle first prediction

print("Predictions completed!")
print(f"Train predictions shape: {train_predictions_scaled.shape}")
print(f"Test predictions shape: {test_predictions_scaled.shape}")
print(f"Baseline predictions shape: {baseline_predictions.shape}")


## 11. Performance Metrics and Baseline Comparison


In [ ]:
def calculate_metrics(actual, predicted, model_name):
    """
    Calculate comprehensive performance metrics
    """
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)

    # Mean Absolute Percentage Error
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100

    # Directional Accuracy
    actual_direction = np.diff(actual.flatten()) > 0
    predicted_direction = np.diff(predicted.flatten()) > 0
    directional_accuracy = np.mean(actual_direction == predicted_direction) * 100

    return {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'Directional_Accuracy': directional_accuracy
    }

# Calculate metrics for both models
lstm_metrics = calculate_metrics(y_test_scaled, test_predictions_scaled, 'LSTM')
baseline_metrics = calculate_metrics(y_test_scaled, baseline_predictions, 'Naive Baseline')

# Create comparison DataFrame
metrics_df = pd.DataFrame([lstm_metrics, baseline_metrics])
metrics_df = metrics_df.round(4)

print("📊 PERFORMANCE COMPARISON")
print("=" * 50)
print(metrics_df.to_string(index=False))

# Calculate improvement over baseline
rmse_improvement = ((baseline_metrics['RMSE'] - lstm_metrics['RMSE']) / baseline_metrics['RMSE']) * 100
directional_improvement = lstm_metrics['Directional_Accuracy'] - baseline_metrics['Directional_Accuracy']

print(f"\n📈 MODEL EFFECTIVENESS")
print("=" * 30)
print(f"RMSE Improvement over Baseline: {rmse_improvement:.2f}%")
print(f"Directional Accuracy Improvement: {directional_improvement:.2f} percentage points")

if rmse_improvement > 10:
    print("✅ LSTM shows significant improvement over naive baseline")
elif rmse_improvement > 0:
    print("✅ LSTM shows modest improvement over naive baseline")
else:
    print("⚠️  LSTM does not outperform naive baseline - consider model refinement")


## 12. Comprehensive Visualization


In [ ]:
# Create comprehensive prediction visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle(f'{STOCK_TICKER} Stock Price Prediction Analysis', fontsize=16, fontweight='bold')

# 1. Full prediction comparison
test_dates = enhanced_data.index[-len(y_test_scaled):]

axes[0, 0].plot(test_dates, y_test_scaled, label='Actual Price', color='blue', linewidth=2)
axes[0, 0].plot(test_dates, test_predictions_scaled, label='LSTM Prediction', color='red', linewidth=2, alpha=0.8)
axes[0, 0].plot(test_dates, baseline_predictions, label='Naive Baseline', color='green', linewidth=1, linestyle='--')
axes[0, 0].set_title('Stock Price Predictions vs Actual')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Prediction errors
lstm_errors = y_test_scaled.flatten() - test_predictions_scaled.flatten()
baseline_errors = y_test_scaled.flatten() - baseline_predictions.flatten()

axes[0, 1].hist(lstm_errors, bins=30, alpha=0.7, label='LSTM Errors', color='red')
axes[0, 1].hist(baseline_errors, bins=30, alpha=0.7, label='Baseline Errors', color='green')
axes[0, 1].set_title('Prediction Error Distribution')
axes[0, 1].set_xlabel('Prediction Error ($)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Scatter plot: Actual vs Predicted
axes[1, 0].scatter(y_test_scaled, test_predictions_scaled, alpha=0.6, color='red', label='LSTM')
axes[1, 0].scatter(y_test_scaled, baseline_predictions, alpha=0.6, color='green', label='Baseline')
min_price = min(y_test_scaled.min(), test_predictions_scaled.min())
max_price = max(y_test_scaled.max(), test_predictions_scaled.max())
axes[1, 0].plot([min_price, max_price], [min_price, max_price], 'k--', alpha=0.8, label='Perfect Prediction')
axes[1, 0].set_title('Actual vs Predicted Prices')
axes[1, 0].set_xlabel('Actual Price ($)')
axes[1, 0].set_ylabel('Predicted Price ($)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Directional accuracy over time
window_size = 20
lstm_directions = np.diff(test_predictions_scaled.flatten()) > 0
actual_directions = np.diff(y_test_scaled.flatten()) > 0
rolling_accuracy = []

for i in range(window_size, len(lstm_directions)):
    window_accuracy = np.mean(lstm_directions[i-window_size:i] == actual_directions[i-window_size:i]) * 100
    rolling_accuracy.append(window_accuracy)

rolling_dates = test_dates[window_size+1:]
axes[1, 1].plot(rolling_dates, rolling_accuracy, color='purple', linewidth=2)
axes[1, 1].axhline(y=50, color='red', linestyle='--', alpha=0.7, label='Random Chance')
axes[1, 1].set_title(f'Rolling Directional Accuracy ({window_size}-day window)')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Save the plot
plt.savefig(f'{STOCK_TICKER}_lstm_analysis.png', dpi=300, bbox_inches='tight')
print(f"\n📊 Analysis chart saved as '{STOCK_TICKER}_lstm_analysis.png'")


## 13. Critical Analysis & Insights

This section provides a thorough evaluation of our LSTM model's performance and practical implications.


In [ ]:
# Detailed performance analysis
print("🔍 CRITICAL ANALYSIS OF LSTM PERFORMANCE")
print("=" * 60)

# 1. Statistical Significance
from scipy import stats

lstm_abs_errors = np.abs(lstm_errors)
baseline_abs_errors = np.abs(baseline_errors)

# Paired t-test to check if LSTM errors are significantly different
t_stat, p_value = stats.ttest_rel(lstm_abs_errors, baseline_abs_errors)

print(f"\n📊 STATISTICAL SIGNIFICANCE")
print(f"Paired t-test p-value: {p_value:.6f}")
if p_value < 0.05:
    print("✅ LSTM performance is statistically significantly different from baseline")
else:
    print("⚠️  LSTM performance is not statistically significantly different from baseline")

# 2. Directional Accuracy Analysis
print(f"\n🎯 DIRECTIONAL ACCURACY BREAKDOWN")
print(f"LSTM Directional Accuracy: {lstm_metrics['Directional_Accuracy']:.2f}%")
print(f"Baseline Directional Accuracy: {baseline_metrics['Directional_Accuracy']:.2f}%")

if lstm_metrics['Directional_Accuracy'] > 55:
    print("✅ Strong directional prediction capability")
elif lstm_metrics['Directional_Accuracy'] > 50:
    print("✅ Modest directional prediction capability")
else:
    print("⚠️  Poor directional prediction - worse than random")

# 3. Volatility Analysis
actual_volatility = np.std(y_test_scaled)
predicted_volatility = np.std(test_predictions_scaled)
volatility_ratio = predicted_volatility / actual_volatility

print(f"\n📈 VOLATILITY ANALYSIS")
print(f"Actual Price Volatility: ${actual_volatility:.2f}")
print(f"Predicted Price Volatility: ${predicted_volatility:.2f}")
print(f"Volatility Ratio (Pred/Actual): {volatility_ratio:.2f}")

if 0.8 <= volatility_ratio <= 1.2:
    print("✅ Model captures volatility well")
elif volatility_ratio < 0.8:
    print("⚠️  Model underestimates volatility (too conservative)")
else:
    print("⚠️  Model overestimates volatility (too aggressive)")

# 4. Trend Following Analysis
y_test_float32 = y_test_scaled.flatten().astype('float32')
predictions_float32 = test_predictions_scaled.flatten().astype('float32')

actual_trend = np.polyfit(range(len(y_test_float32)), y_test_float32, 1)[0]
predicted_trend = np.polyfit(range(len(predictions_float32)), predictions_float32, 1)[0]

print(f"\n📊 TREND ANALYSIS")
print(f"Actual Trend ($/day): {actual_trend:.4f}")
print(f"Predicted Trend ($/day): {predicted_trend:.4f}")
print(f"Trend Capture Ratio: {predicted_trend/actual_trend:.2f}" if actual_trend != 0 else "Trend Capture: N/A (flat trend)")

# 5. Error Analysis by Market Conditions
price_changes = np.diff(y_test_scaled.flatten())
up_days = price_changes > 0
down_days = price_changes < 0

if len(price_changes) > 1:
    up_day_errors = lstm_errors[1:][up_days]
    down_day_errors = lstm_errors[1:][down_days]

    print(f"\n📊 PERFORMANCE BY MARKET CONDITION")
    if len(up_day_errors) > 0:
        print(f"Average error on up days: ${np.mean(np.abs(up_day_errors)):.2f}")
    if len(down_day_errors) > 0:
        print(f"Average error on down days: ${np.mean(np.abs(down_day_errors)):.2f}")

print("\n" + "=" * 60)


## 14. Model Limitations & Real-World Considerations

### 🚨 **Critical Limitations**

1. **Historical Bias**: The model is trained exclusively on past data and assumes historical patterns will continue

2. **Black Swan Events**: Cannot predict unprecedented market events (crashes, pandemics, geopolitical crises)

3. **Market Regime Changes**: May fail when market dynamics fundamentally shift

4. **Feature Limitations**: Only uses price and volume data - ignores fundamental analysis, news sentiment, macroeconomic factors

5. **Overfitting Risk**: Complex models may memorize noise rather than learn genuine patterns

6. **Transaction Costs**: Real trading involves spreads, commissions, and slippage not accounted for in predictions

7. **Market Impact**: Large trades based on model predictions could move prices, invalidating the predictions

### ⚖️ **Regulatory and Ethical Considerations**

- **Not Financial Advice**: This model is for educational purposes only
- **Risk Management**: Never risk more than you can afford to lose
- **Diversification**: Don't rely on a single model or asset
- **Continuous Monitoring**: Model performance can degrade over time


## 15. Next Steps for Model Improvement

### 🔧 **Immediate Improvements**

1. **Hyperparameter Tuning**:
   - Grid search for optimal LSTM units, dropout rates, learning rates
   - Experiment with different sequence lengths
   - Try different optimizers (RMSprop, AdaGrad)

2. **Alternative Architectures**:
   - **GRU (Gated Recurrent Unit)**: Often performs similarly to LSTM with fewer parameters
   - **Bidirectional LSTM**: Processes sequences in both directions
   - **Attention Mechanisms**: Focus on most relevant time steps
   - **Transformer Models**: State-of-the-art for sequence modeling

3. **Enhanced Features**:
   - **More Technical Indicators**: MACD, Bollinger Bands, Stochastic Oscillator
   - **Market Sentiment**: VIX (fear index), put/call ratios
   - **Fundamental Data**: P/E ratios, earnings, revenue growth
   - **Macroeconomic Indicators**: Interest rates, inflation, GDP growth

### 🚀 **Advanced Enhancements**

4. **Multi-Asset Models**:
   - Include correlated assets (sector ETFs, commodities)
   - Cross-asset attention mechanisms

5. **News and Sentiment Integration**:
   - Natural Language Processing on financial news
   - Social media sentiment analysis
   - Earnings call transcripts analysis

6. **Ensemble Methods**:
   - Combine multiple models (LSTM + Random Forest + Linear Regression)
   - Weighted voting based on recent performance

7. **Online Learning**:
   - Continuously update model with new data
   - Adaptive learning rates based on market conditions

### 📊 **Validation Improvements**

8. **Walk-Forward Validation**:
   - Multiple train-test cycles
   - Out-of-sample testing across different market conditions

9. **Risk-Adjusted Metrics**:
   - Sharpe ratio, Sortino ratio
   - Maximum drawdown analysis
   - Value at Risk (VaR) calculations


In [ ]:
# Final summary and recommendations
print("🎯 FINAL RECOMMENDATIONS")
print("=" * 40)

if rmse_improvement > 15 and lstm_metrics['Directional_Accuracy'] > 55:
    print("✅ STRONG MODEL: Consider for further development")
    print("   → Focus on hyperparameter tuning and feature engineering")
    print("   → Implement walk-forward validation")
    print("   → Add risk management components")

elif rmse_improvement > 5 and lstm_metrics['Directional_Accuracy'] > 50:
    print("✅ PROMISING MODEL: Needs refinement")
    print("   → Try alternative architectures (GRU, Attention)")
    print("   → Add more features (sentiment, fundamentals)")
    print("   → Implement ensemble methods")

else:
    print("⚠️  WEAK MODEL: Significant improvements needed")
    print("   → Reconsider feature selection")
    print("   → Try completely different approaches")
    print("   → Consider if this asset is predictable with current methods")

print(f"\n📈 MODEL PERFORMANCE SUMMARY FOR {STOCK_TICKER}")
print(f"   RMSE: ${lstm_metrics['RMSE']:.2f}")
print(f"   Directional Accuracy: {lstm_metrics['Directional_Accuracy']:.1f}%")
print(f"   Improvement over Baseline: {rmse_improvement:.1f}%")

print("\n⚠️  REMEMBER: This is for educational purposes only.")
print("   Always conduct thorough backtesting and risk assessment")
print("   before considering any real-world application.")

print("\n🎉 Analysis Complete! Thank you for using this comprehensive LSTM stock forecasting notebook.")
